In [11]:
import warnings

warnings.filterwarnings("ignore")

In [12]:
from flowermd.library import EllipsoidChain, EllipsoidForcefield
from flowermd import Pack, Simulation
from flowermd.utils import create_rigid_ellipsoid_chain, get_target_box_number_density
import unyt as u
from cmeutils.gsd_utils import ellipsoid_gsd
import gsd.hoomd

In [13]:
ellipsoids = EllipsoidChain(lengths=1, num_mols=128, bead_mass=1.0, lpar=1.0)

In [14]:
system = Pack(
    density=0.15*u.Unit("nm**-3"),
    molecules=ellipsoids,
    packing_expand_factor=6,
    edge=2, overlap=1,
    fix_orientation=True
)

In [15]:
forcefield = EllipsoidForcefield(
    bond_k=100, bond_r0=0,
    epsilon=1.0,
    lpar=1.0,
    lperp=0.5,
    r_cut=2.0
)
rigid_frame, rigid_constraint = create_rigid_ellipsoid_chain(snapshot=system.hoomd_snapshot)

In [16]:
sim = Simulation(
    constraint=rigid_constraint,
    forcefield=forcefield.hoomd_forces,
    gsd_file_name="ellipsoid_overlap.gsd",
    log_file_name="ellipsoid_overlap.txt",
    initial_state=rigid_frame,
    gsd_write_freq=100
)

Initializing simulation state from a gsd.hoomd.Frame.


Initial Parameters:
 - density: 1.0
 - period: 10
 - n_steps: 6e4
 - kT: 7.0
 - tau_kt: 100*sim.dt
 - thermalize_particles: True

In [17]:
target_box = get_target_box_number_density(1.0*u.Unit("nm**-3"), n_beads=128)
sim.run_update_volume(
    final_box_lengths=target_box,
    period=10,
    n_steps=6e4,
    kT=7.0,
    tau_kt=100*sim.dt,
    thermalize_particles=True,
)

Step 550 of 60000; TPS: 1468.07; ETA: 0.7 minutes
Step 1100 of 60000; TPS: 2179.69; ETA: 0.5 minutes
Step 1650 of 60000; TPS: 2714.68; ETA: 0.4 minutes
Step 2200 of 60000; TPS: 3167.74; ETA: 0.3 minutes
Step 2750 of 60000; TPS: 3515.59; ETA: 0.3 minutes
Step 3300 of 60000; TPS: 3744.6; ETA: 0.3 minutes
Step 3850 of 60000; TPS: 3979.81; ETA: 0.2 minutes
Step 4400 of 60000; TPS: 4180.17; ETA: 0.2 minutes
Step 4950 of 60000; TPS: 4311.4; ETA: 0.2 minutes
Step 5500 of 60000; TPS: 4458.3; ETA: 0.2 minutes
Step 6050 of 60000; TPS: 4588.58; ETA: 0.2 minutes
Step 6600 of 60000; TPS: 4670.78; ETA: 0.2 minutes
Step 7150 of 60000; TPS: 4772.92; ETA: 0.2 minutes
Step 7700 of 60000; TPS: 4858.32; ETA: 0.2 minutes
Step 8250 of 60000; TPS: 4913.68; ETA: 0.2 minutes
Step 8800 of 60000; TPS: 4981.75; ETA: 0.2 minutes
Step 9350 of 60000; TPS: 5045.75; ETA: 0.2 minutes
Step 9900 of 60000; TPS: 5082.95; ETA: 0.2 minutes
Step 10450 of 60000; TPS: 5134.34; ETA: 0.2 minutes
Step 11000 of 60000; TPS: 5183.33;

In [18]:
sim.run_NVT(n_steps=2e4, kT=1.0, tau_kt=0.001)
sim.flush_writers()


Step 499 of 20000; TPS: 737.05; ETA: 0.4 minutes
Step 1049 of 20000; TPS: 728.96; ETA: 0.4 minutes
Step 1599 of 20000; TPS: 741.15; ETA: 0.4 minutes
Step 2149 of 20000; TPS: 737.34; ETA: 0.4 minutes
Step 2699 of 20000; TPS: 741.58; ETA: 0.4 minutes
Step 3249 of 20000; TPS: 735.5; ETA: 0.4 minutes
Step 3799 of 20000; TPS: 738.15; ETA: 0.4 minutes
Step 4349 of 20000; TPS: 738.13; ETA: 0.4 minutes
Step 4899 of 20000; TPS: 737.6; ETA: 0.3 minutes
Step 5449 of 20000; TPS: 738.53; ETA: 0.3 minutes
Step 5999 of 20000; TPS: 740.12; ETA: 0.3 minutes
Step 6549 of 20000; TPS: 738.78; ETA: 0.3 minutes
Step 7099 of 20000; TPS: 739.72; ETA: 0.3 minutes
Step 7649 of 20000; TPS: 740.74; ETA: 0.3 minutes
Step 8199 of 20000; TPS: 740.84; ETA: 0.3 minutes
Step 8749 of 20000; TPS: 740.63; ETA: 0.3 minutes
Step 9299 of 20000; TPS: 741.57; ETA: 0.2 minutes
Step 9849 of 20000; TPS: 741.94; ETA: 0.2 minutes
Step 10399 of 20000; TPS: 742.03; ETA: 0.2 minutes
Step 10949 of 20000; TPS: 742.06; ETA: 0.2 minutes
S

In [19]:
ellipsoid_gsd_better(gsd_file="ellipsoid_overlap.gsd", new_file="ovito-ellipsoid_overlap.gsd", ellipsoid_types='R', lpar=1.0, lperp=0.5)

In [9]:
def ellipsoid_gsd_better(gsd_file, new_file, ellipsoid_types, lpar, lperp):
    """Add needed information to GSD file to visualize ellipsoids.

    Saves a new GSD file with lpar and lperp values populated
    for each particle. Ovito can be used to visualize the new GSD file.

    Parameters
    ----------
    gsd_file : str
        Path to the original GSD file containing trajectory information
    new_file : str
        Path and filename of the new GSD file
    ellipsoid_types : str or list of str
        The particle types (i.e. names) of particles to be drawn
        as ellipsoids.
    lpar : float
        Value of lpar of the ellipsoids
    lperp : float
        Value of lperp of the ellipsoids

    """
    with gsd.hoomd.open(new_file, "w") as new_t:
        with gsd.hoomd.open(gsd_file) as old_t:
            for snap in old_t:
                shape_dicts_list = []
                for ptype in snap.particles.types:
                    if ptype == ellipsoid_types or ptype in ellipsoid_types:
                        shapes_dict = {
                            "type": "Ellipsoid",
                            "a": lpar,
                            "b": lperp,
                            "c": lperp,
                        }
                    else:
                        shapes_dict = {"type": "Sphere", "diameter": 0.001}
                    shape_dicts_list.append(shapes_dict)
                snap.particles.type_shapes = shape_dicts_list
                snap.validate()
                new_t.append(snap)